# 검색 알고리즘 실험
> 특정 문서를 입력받아 코사인 유사도를 기반으로 유사한 문서를 반환하는 실험을 진행해보자.

1. TF-IDF + NMF
1. Word2vec
1. Fasttext

## Import

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.preprocessing import Normalizer
from konlpy.tag import Mecab

## Data
>sourced by momtalk(https://www.momtalk.kr/talk/list/all)

1. `df1`: 육아정보 포스팅
1. `df2`: 유아용품 포스팅
1. `df` : 위의 두 데이터를 합친 것

In [2]:
df1 = pd.read_csv("data/maeili.csv", encoding='utf-8')
df2 = pd.read_csv("data/baby.csv", encoding='utf-8')
df3 = pd.read_csv("data/toys.csv", encoding='utf-8')
df4 = pd.read_csv("data/seoulchildcare.csv", encoding='utf-8')
df5 = pd.read_csv("data/babynews_DLC.csv", encoding='utf-8')
df6 = pd.read_csv("data/ildongfoodis.csv", encoding='utf-8')
df7 = pd.read_csv("data/momsmagazine.csv", encoding='utf-8')
df8 = pd.read_csv("data/ange.csv", encoding='utf-8')

df = pd.concat([df1.text, df2.text, df3.text, df4.text, 
                df5.text, df6.text, df7.text, df8.text])
df = df.drop(index=[952, 955, 956, 960, 962, 964, 1165, 1240, 1322])
df = df.dropna()
df = pd.DataFrame(df).reset_index(drop=True)
df.tail()

,text
17301,아이는 현재 신체와 정신이 모두 발달 중에 있다 신체 발달은 근육의 발달을 정신 발...
17302,세 살 적 버릇이 여든까지 간다는 말이 있다 밥상머리 예절도 예외는 아니다 아이에게...
17303,아이의 키가 자라고 몸무게가 늘어나면서 아이 몸을 구성하고 있는 장기들도 커지고 무...
17304,눈치는 직관적이고 비언어적인 방법으로 의사소통을 하고 문제를 해결하는 놀라운 능력이...
17305,소리를 들으면 사람은 그 소리를 그대로 흉내 낼 수 있다 이것이 청력의 힘이다 말소...


## TF-IDF

In [3]:
def tokenize(text):
    mecab = Mecab()
    tokens = mecab.pos(text)
    key_tokens = [token[0] for token in tokens 
                 if ('NNG' in token or 'NNP' in token or 'VA' in token or 'VV' in token)]
    key_tokens = ' '.join(key_tokens)
    return key_tokens

    
df['token'] = df.text.apply(tokenize)

In [4]:
df.tail()

,text,token
17301,아이는 현재 신체와 정신이 모두 발달 중에 있다 신체 발달은 근육의 발달을 정신 발...,아이 신체 정신 발달 있 신체 발달 근육 발달 정신 발달 지적 호기심 발달 포함 공...
17302,세 살 적 버릇이 여든까지 간다는 말이 있다 밥상머리 예절도 예외는 아니다 아이에게...,버릇 말 있 밥상머리 예절 예외 아이 식습관 가족 밥상 앞 앉 우선 가족 자신 자리...
17303,아이의 키가 자라고 몸무게가 늘어나면서 아이 몸을 구성하고 있는 장기들도 커지고 무...,아이 키 자라 몸무게 늘어나 아이 몸 구성 장기 무게 성장 발달 신체 부분 일정 순...
17304,눈치는 직관적이고 비언어적인 방법으로 의사소통을 하고 문제를 해결하는 놀라운 능력이...,눈치 직관 언어 방법 의사소통 하 문제 해결 능력 사람 관계 맺 사람 생각 감정 이...
17305,소리를 들으면 사람은 그 소리를 그대로 흉내 낼 수 있다 이것이 청력의 힘이다 말소...,소리 사람 소리 흉내 있 청력 힘 말 소리 아이 듣 소리 모방 소리 소리 감정 느낌...


In [5]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(df['token'])

print(vectors.shape)

(17306, 52618)


## NMF(Non-negative Matrix Factorization)
- sparse coding처럼 dictionary와 encoded vector 학습
- parameter가 0보다 큰 특성으로 feature matrix가 원본 데이터들과 어떠한 관계를 가지는지 해석이 가능
- 연산에 약간의 시간 소요

In [6]:
%%time
vector_array = vectors.toarray()
nmf = NMF(n_components=40)
nmf.fit(vector_array)
features = nmf.transform(vector_array)

CPU times: user 15min 9s, sys: 8.1 s, total: 15min 17s
Wall time: 2min 1s


## Normalize extracted feature set
- 0과 1사이의 값으로 정규화하여 feature vector의 길이를 1로 고정

In [7]:
from sklearn.preprocessing import Normalizer

normalizer = Normalizer()
norm_features=normalizer.fit_transform(features)

print(norm_features[0:2])

[[0.71261336 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.00389628 0.         0.         0.         0.592812
  0.20593723 0.14062457 0.         0.         0.         0.18695898
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.18449045 0.09780161 0.
  0.         0.         0.         0.        ]
 [0.50994692 0.         0.         0.10420228 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.81103146 0.         0.         0.         0.01620445 0.16409557
  0.         0.         0.14205294 0.         0.         0.06022933
  0.0918441  0.06572892 0.         0.08569398 0.         0.0151111
  0.         0.         0.         0.        ]]


In [8]:
df_features = pd.DataFrame(norm_features,index=df.index.tolist())
df_features.tail()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
17301,0.469231,0.0,0.000000,0.003950,0.023671,0.000000,0.0,0.000000,0.000000,0.067784,...,0.088754,0.000000,0.03851,0.000000,0.000000,0.000000,0.019101,0.0,0.0,0.017121
17302,0.651384,0.0,0.000000,0.034809,0.000000,0.000000,0.0,0.000000,0.000000,0.448324,...,0.016399,0.300472,0.00000,0.131928,0.000000,0.000000,0.118490,0.0,0.0,0.000000
17303,0.150421,0.0,0.023623,0.024743,0.000000,0.000000,0.0,0.010280,0.000000,0.000000,...,0.183334,0.000000,0.00000,0.005918,0.000000,0.053625,0.000000,0.0,0.0,0.000000
17304,0.428456,0.0,0.000000,0.188804,0.003426,0.006913,0.0,0.028218,0.013103,0.000000,...,0.000000,0.027621,0.00000,0.000000,0.039106,0.000000,0.000000,0.0,0.0,0.000000
17305,0.026630,0.0,0.000000,0.026475,0.000000,0.121803,0.0,0.000000,0.000000,0.000000,...,0.065552,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000


`df`의 맨 첫번째 텍스트를 정규화 한 40차원 벡터의 값

In [9]:
print(df.loc[0].text)
article = df_features.loc[0]
len(article)

6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요  매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다 


40

In [10]:
similarities=df_features.dot(article)
top=similarities.nlargest(10)

texts = df.loc[top.index]['text'].tolist()
i = 0
for text in texts:
    df_idx = top.index[i]
    print('TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
    #print(text+'\n')
    i = i+1

TITLE :0 Similarities:1.0
TITLE :1788 Similarities:0.9408881812088815
TITLE :869 Similarities:0.9378332493233649
TITLE :14011 Similarities:0.9231710618772622
TITLE :1801 Similarities:0.9124326341052716
TITLE :260 Similarities:0.9096111881462734
TITLE :11623 Similarities:0.907023712152378
TITLE :5811 Similarities:0.9062118106424178
TITLE :8867 Similarities:0.8990816489524339
TITLE :9830 Similarities:0.8970800266271007


In [11]:
i = 0
for text in texts:
    df_idx = top.index[i]
    print('TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
    print(text+'\n')
    i = i+1

TITLE :0 Similarities:1.0
6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요  매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다 

TITLE :1788 Similarities:0.9408881812088815
 아이가 갑자기 학원을 안 가겠대요 이미지 아이아트그만둬도 될까 아이가 갑자기 학원을 안 가겠대요 아이가 너무 원해서 발레학원을 다니기 

---

## Word2Vec
- 위의 TF-IDF 임베딩이 아닌 Word2Vec 임베딩으로 실험해보자.
- 음수의 값이 포함되서는 안되는 NMF는 활용할 수 없다. 어차피 sparse하지 않으면서 TF-IDF에 비해 훨씬 저차원인 성격을 지닌 word2vec 임베딩은 바로 유사도를 계산할 수 있어 더욱 편리하게 활용할 수 있을 것이라 판단된다.

In [12]:
from gensim.models import Word2Vec
from konlpy.tag import Mecab
import pandas as pd
import numpy as np

In [13]:
w2v_model = Word2Vec.load('data/tokmom_word2vec_all')
df = pd.read_csv('data/text_for_word2vec.csv')
df = df.drop(index=df[df['text'] == ""].index)
df = df.dropna()
df = df.reset_index(drop=True)

In [14]:
%%time
def tokenize(text):
    tokenizer = Mecab()
    text = tokenizer.morphs(text)
    return text
    
df['tokenized_text'] = df['text'].apply(tokenize)

CPU times: user 38.4 s, sys: 19.6 s, total: 58 s
Wall time: 58 s


In [15]:
def make_to_vector(tokens):
    mean_of_vectors = []
    except_tokens = []
    for token in tokens:
        try:
            vector = w2v_model.wv.get_vector(token)
            mean_of_vectors.append(vector)
        except KeyError:
#             print("{} is not exist in the vocab".format(noun))
            except_tokens.append(token)
            continue
    try:
        mean_of_vectors = sum(mean_of_vectors) / len(mean_of_vectors)
        return mean_of_vectors
    except ZeroDivisionError:
        return np.nan

In [16]:
df['doc_vector'] = df['tokenized_text'].apply(make_to_vector)
df = df.dropna()
df = df.reset_index(drop=True)

In [17]:
vectors = np.zeros(128,)
for vector in df.doc_vector:
    vectors = np.vstack([vectors, vector])
vectors = vectors[1:, :]
vectors

array([[-0.04137425,  0.3251076 ,  0.18798998, ...,  0.08232684,
         0.02341757,  0.11592384],
       [-0.00112048,  0.3264688 ,  0.17844217, ...,  0.07640506,
         0.08945058,  0.15669847],
       [-0.03452514,  0.33918482,  0.19525182, ...,  0.09566827,
         0.04590433,  0.1497242 ],
       ...,
       [-0.04784118,  0.34894699,  0.3020418 , ...,  0.12919152,
         0.12997888,  0.12646754],
       [ 0.01710394,  0.30797645,  0.1927827 , ...,  0.06712007,
         0.12091841,  0.12804736],
       [-0.00528116,  0.36075997,  0.19418404, ...,  0.075033  ,
         0.12559994,  0.09478852]])

>ndarray 형식으로 예쁘게 만들어야 되는데 이런 식으로 밖에 못만드는게 내 실력 미달인듯... ㅠ

## Normalize extracted feature set
- 유사도 계산의 편의를 위해 0과 1사이의 값으로 정규화

In [18]:
from sklearn.preprocessing import Normalizer

normalizer = Normalizer()
norm_features=normalizer.fit_transform(vectors)
df_features = pd.DataFrame(norm_features,index=df.index.tolist())
df_features.tail()

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
16260,-0.002461,0.183983,0.141804,-0.046512,-0.070350,-0.041932,-0.181018,0.030576,0.042004,-0.072899,...,0.055147,0.017877,-0.119578,-0.132036,-0.019952,0.001163,-0.005893,0.060095,0.052588,0.058712
16261,-0.009040,0.194548,0.147878,-0.092404,-0.046953,-0.052826,-0.241600,0.022109,0.020520,-0.100588,...,0.134004,0.015010,-0.135623,-0.114320,0.001728,0.001734,-0.023712,0.070371,0.071354,0.093451
16262,-0.027308,0.199177,0.172404,-0.101010,-0.079994,-0.055174,-0.213246,0.054337,0.044420,-0.075429,...,0.084699,0.007715,-0.125904,-0.121061,-0.015941,0.026472,-0.019395,0.073742,0.074191,0.072187
16263,0.010430,0.187799,0.117556,-0.086350,-0.063616,-0.049801,-0.217499,0.014444,-0.001868,-0.039030,...,0.093132,0.016238,-0.102829,-0.128487,-0.003526,0.005309,0.003900,0.040929,0.073734,0.078081
16264,-0.003224,0.220218,0.118536,-0.079401,-0.069805,-0.047057,-0.234342,0.026555,0.009296,-0.043136,...,0.073331,0.004431,-0.094670,-0.100207,-0.019792,0.008163,0.008016,0.045802,0.076670,0.057862


In [19]:
article = df_features.loc[0]
similarities=df_features.dot(article) # cosine similarity
top=similarities.nlargest(5)

texts = df.loc[top.index]['text'].tolist()

i = 0
print("<비교할 문서>")
for text in texts:
    df_idx = top.index[i]
    print('TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
    print(text+'\n')
    i = i+1

<비교할 문서>
TITLE :0 Similarities:1.0000000000000004
6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요 매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다

TITLE :6948 Similarities:0.9896809968340001
아이가 앵무새처럼 말을 곧잘 따라 하니 자꾸 확인하고 싶고 물어보고 싶은 게 많아진다 특히 예나 

### 가상의 키워드를 입력하여 유사도 계산해보기
- 이번에는 챗봇처럼 짧은 문장(아마 키워드 위주)을 입력받아 유사도 계산을 해보자.
- word2vec vocabulary에 없는 단어를 입력받을 경우 에러가 발생하는 문제가 있다.

In [20]:
tokenizer = Mecab()

while True:
    input_text = input("Input text:")
    if input_text == "exit":
        print("Bye")
        break
        
    tokens = tokenizer.morphs(input_text)

    try:
        input_vector = [w2v_model.wv.get_vector(token) for token in tokens]
        input_vector = normalizer.fit_transform(input_vector)
        input_vector = sum(input_vector) / len(input_vector)
        

        similarities=df_features.dot(input_vector)
        top=similarities.nlargest(5)

        texts = df.loc[top.index]['text'].tolist()

        i = 0
        for text in texts:
            df_idx = top.index[i]
            print('\n'+'TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
            print(text+'\n')
            i = i+1
            
    except:
        print("'{}'은(는) 제가 아직 잘 이해하지 못하겠어요ㅠㅠ ".format(input_text))


Input text:시댁 갈등

TITLE :15564 Similarities:0.5267278508012525
사위 사랑은 장모라는 옛말이 무색하게 최근에는 고부 갈등보다장모와 사위 즉 장서 갈등으로 고민하는 경우가 늘고 있다 특히육아맘의 경우 임신과 출산 육아를 불편한 시댁보다 친정에 도움받는 일이 잦아지면서 장서 갈등이 더 많아지고 있다 장모 입장에서는 애지중지 기른 귀한 딸이 고생하는 게 안쓰럽고 자신보다 더나은 삶을 살길 바라는 마음이 크고 사위 입장에서는 자녀 양육이라는 이유로 장모와 부딪치는 시간이 많아지고 장모의 행동이 결혼 생활에까지 영향을 끼친다는 생각에 불만이 늘면서 갈등이 커지는 것이다 또한 중간자인 아내가 지나치게 친정엄마에게 의존하거나 남편과의 관계가 원만하지 않을 경우에도 장서 갈등이 더심각해질 수 있다


TITLE :16097 Similarities:0.5108468539198052
결혼한 여자에게 명절 연휴는 꼭 달갑지만은 않다 오랜만에 가족들을 만날 생각에 설레기도 하지만 부모님 용돈에 선물 온갖 집안일 등이 기다리는 데다 가족 갈등이 생길 가능성도 높아지기때문이다 실제 한 취업 포털에서 기혼 직장인을 대상으로 명절 부부 싸움에 대해 설문 조사를실시한 결과 부부 싸움을 한다가 70에 달할 만큼 명절 때 부부 싸움이 잦았다시가와 친정 양가 가족으로 인한 갈등도 깊어지고 이 문제에 대한 부부의 의견이 충돌하는 경우가 많은데 대개는 잘 조율하지 못하기 때문에 작지 않은 부부 싸움으로 번진다 싸움을 예방하고 대처하는 처세술이 중요하다 case1 명절 일은 며느리의 몫 남편은 일도 안 도와주면서명절 음식 준비며 치우는 것은 다 며느리 몫이라고생각해요 손끝 하나 까딱 안 하는 남편 때문에 너무 화가 납니다 명절이 끝나고 불만을 얘기하면 늘 싸움으로 이어지니 명절이란 소리만 들어도 스트레스를 받아요 남편의 처세술 남편은 명절 계획과 준비 과정 등을 아내와 상의하고 역할을 어떻게 분담하는 게 좋을지 아내에게 먼저 의견을 물어봤어야 한다 무엇

## Fasttext

In [21]:
from gensim.models import FastText

In [22]:
fasttext_model = FastText.load('data/tokmom_fasttext.model')
df = pd.read_csv('data/text_for_word2vec.csv')
df = df.dropna()
df = df.reset_index(drop=True)

### Preprocessing for FastText 
- 한글을 자음, 모음 단위로 분해
- 단어 단위로 구분된 ndarray 자료구조로 정제

In [23]:
import re
from soynlp.hangle import decompose, compose

def remove_doublespace(s):
    doublespace_pattern = re.compile('\s+')
    return doublespace_pattern.sub(' ', s).strip()

def encode(s):
    def process(c):
        if c == ' ':
            return c
        jamo = decompose(c)
        # 'a' or 모음 or 자음
        if (jamo is None) or (jamo[0] == ' ') or (jamo[1] == ' '):
            return ' '
        base = jamo[0]+jamo[1]
        if jamo[2] == ' ':
            return base + '-'
        return base + jamo[2]

    s = ''.join(process(c) for c in s)
    return remove_doublespace(s).strip()

#     s = [process(c) for c in s]
#     return s

def decode(s):
    def process(t):
        assert len(t) % 3 == 0
        t_ = t.replace('-', ' ')
        chars = [tuple(t_[3*i:3*(i+1)]) for i in range(len(t_)//3)]
        recovered = [compose(*char) for char in chars]
        recovered = ''.join(recovered)
        return recovered

    return ' '.join(process(t) for t in s.split())

In [24]:
df['encoded'] = df['text'].apply(encode)

### Vectorizing

In [25]:
def jamo_to_vector(tokens):
    mean_of_vectors = []
    except_tokens = []
    tokens = tokens.split(' ') # 띄어쓰기 단위로 글자 구분을 해준다.
    for token in tokens:
        try:
            vector = fasttext_model.wv.get_vector(token)
            mean_of_vectors.append(vector)
        except KeyError:
            print("{} is not exist in the vocab".format(noun))
            except_tokens.append(token)
            continue
    try:
        mean_of_vectors = sum(mean_of_vectors) / len(mean_of_vectors)
        return mean_of_vectors
    except ZeroDivisionError:
        return np.nan

In [26]:
%%time
df['doc_vector'] = df['encoded'].apply(jamo_to_vector)
df = df.dropna()
df = df.reset_index(drop=True)

CPU times: user 44.6 s, sys: 3.38 ms, total: 44.6 s
Wall time: 44.6 s


In [27]:
vectors = np.zeros(100,)
for vector in df.doc_vector:
    vectors = np.vstack([vectors, vector])
vectors = vectors[1:, :]
vectors

array([[ 0.14245233, -0.16667289,  0.56430107, ..., -0.14386092,
        -0.22880173,  0.67283535],
       [ 0.00848483,  0.22363676,  0.43951243, ..., -0.17781258,
        -0.53633583,  0.35333008],
       [-0.00775058,  0.13586167,  0.62959534, ..., -0.37251776,
        -0.29568937,  0.25651363],
       ...,
       [-0.9256714 , -0.67917722,  0.51798522, ..., -0.074491  ,
        -0.23818044,  0.06910886],
       [-0.32897386,  0.04888401,  0.31564313, ..., -0.16938837,
        -0.34065342, -0.04117135],
       [-0.20526057, -0.10729782,  0.45867062, ..., -0.31659016,
        -0.13415039, -0.03369758]])

>예쁘게 만들어야 되는데 이런 식으로 밖에 못만드는게 내 실력 미달인듯... ㅠ

## Normalize extracted feature set
- 유사도 계산의 편의를 위해 0과 1사이의 값으로 정규화

In [28]:
from sklearn.preprocessing import Normalizer

normalizer = Normalizer()
norm_features=normalizer.fit_transform(vectors)
df_features = pd.DataFrame(norm_features,index=df.index.tolist())
df_features.tail()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
16260,-0.147653,-0.133556,0.159467,-0.127439,0.208392,-0.096452,-0.020623,0.116671,0.072690,0.150632,...,-0.103727,0.062585,0.051177,-0.009801,-0.069140,0.000093,-0.109281,-0.002031,0.054882,-0.024976
16261,-0.146029,-0.123515,0.104668,-0.065550,0.202542,-0.115238,-0.110571,0.159174,0.002952,0.047077,...,-0.027280,0.147202,0.125017,-0.040231,-0.128009,-0.040235,-0.119534,0.041557,-0.014264,0.039461
16262,-0.157195,-0.115336,0.087963,-0.059281,0.207278,-0.129734,-0.064696,0.199199,0.026108,0.081923,...,-0.047901,0.153927,0.105408,-0.042383,-0.049402,-0.023996,-0.075713,-0.012650,-0.040447,0.011736
16263,-0.054762,0.008137,0.052543,-0.155648,0.158992,-0.039132,0.043685,0.068720,0.090423,0.106459,...,-0.052116,0.017313,0.076416,0.049678,-0.017656,-0.034907,-0.193022,-0.028197,-0.056706,-0.006854
16264,-0.034281,-0.017920,0.076604,-0.136019,0.143527,-0.000870,0.013069,0.121190,0.089617,0.125199,...,-0.087797,0.041822,0.049803,0.052652,-0.006293,-0.011663,-0.151762,-0.052874,-0.022405,-0.005628


In [29]:
article = df_features.loc[0]
similarities=df_features.dot(article) # cosine similarity
top=similarities.nlargest(5)

texts = df.loc[top.index]['text'].tolist()

i = 0
print("<비교할 문서>")
for text in texts:
    df_idx = top.index[i]
    print('TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
    print(text+'\n')
    i = i+1

<비교할 문서>
TITLE :0 Similarities:0.9999999999999998
6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요 매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다

TITLE :2143 Similarities:0.9464077725754241
워킹맘의 고민 엄마아빠보다 할머니를 더 좋아하는 우리 아이 어떡하죠 워킹맘의 육아 고민할머니가 봐

### 가상의 키워드를 입력하여 유사도 계산해보기
- 이번에는 챗봇처럼 짧은 문장(아마 키워드 위주)을 입력받아 유사도 계산을 해보자.
- Fasttext는 word2vec과 달리 단어를 입력받을 경우 에러가 발생하는 문제를 방지할 수 있다!

In [30]:
while True:
    input_text = input("Input text:")
    if input_text == "exit":
        print("Bye")
        break
        
    tokens = encode(input_text)
    tokens = tokens.split(' ')

#     try:
    input_vector = [fasttext_model.wv.get_vector(token) for token in tokens]
    input_vector = normalizer.fit_transform(input_vector)
    input_vector = (sum(input_vector) / len(input_vector))
    

    similarities=df_features.dot(input_vector)
    top=similarities.nlargest(5)

    texts = df.loc[top.index]['text'].tolist()

    i = 0
    for text in texts:
        df_idx = top.index[i]
        print('\n'+'TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
        print(text+'\n')
        i = i+1
            
#     except:
#         print("'{}'은(는) 제가 아직 잘 이해하지 못하겠어요ㅠㅠ ".format(input_text))


Input text:시댁 갈등

TITLE :11369 Similarities:0.43551799010713227
전라남도는 5일 순천만 국가정원 습지센터에서 환경의 소중함을 일깨우는 에너지절약 체험 환경백일장 등 제22회 환경의 날 기념행사를 열었다 환경의 날 기념행사는 지역 주민 민간 환경단체를 비롯해 김갑섭 전라남도지사 권한대행 행정부지사 천제영 순천부시장 김호남 전라남도지속가능발전협의회 공동의장 박기영 그린순천21추진협의회 상임의장 도의원 시의원 등 300여 명이 참석한 가운데 유공자 표창 기념식 환경보전 퍼포먼스 등으로 진행됐다 특히 순천 아고라 해오름 사물단 공연 온실가스 1인 1톤 줄이기 홍보 동영상 상영 부대행사로 환경의 소중함을 일깨우는 아름다운 자연풍경 환경사진 전시 환경백일장 및 그림 그리기 대회 등이 펼쳐졌다 또한 보건환경연구원 환경산업진흥원 한국에너지공단과 지속가능발전협의회 기후환경네트워크 그린순천21추진협의회 등 환경단체들은 체험부스를 운영했다


TITLE :7761 Similarities:0.4337753754433866
서울 구로구구청장 이성에서는 주민들이 함께 방과후 아이들을 돌본다 오후 1시부터 저녁 7시까지 집 근처 작은도서관 마을활력소 등 주민공간에서 주민들이 학교를 마친 아이들에게 놀이 독서 체험지도 등 다양한 프로그램을 제공한다 구로구는 구로형 아이돌봄체계를 구축해 올해 30개소를 개소하고 2020년까지 50개소로 확대할 것이라고 8일 밝혔다 구로형 아이돌봄체계는 오후 1시부터 저녁 7시까지 놀이지도 독서지도 체험교육부터 학원 챙겨보내기 등의 프로그램을 제공한다 센터 이용 아동은 기관별 15명 정도며 구청은 이를 위한 운영비를 지원한다 운영기관 선정은 돌봄에 대한 학부모 수요 아동청소년에게 유해한 환경이 있는지 여부 인근 돌봄기관 운영 유무 접근성 돌봄 공간 확보 아동 지도 프로그램 돌봄인력 자격 등을 고려해 결정한다 현재 구로구는 시범운영을 위한 공모를 통해 최근 작은도서관 8곳과 마을기업 1곳 등 총 9개 기관을 선

In [31]:
from konlpy.tag import Mecab

tokenizer = Mecab()

In [65]:
def save_keyword(text):
    '''
    입력받은 텍스트에서 명사, 동사, 형용사를 추출한 리스트를 반환합니다.
    '''
    pos_text = tokenizer.morphs(text)
#     words = [w for w in pos_text
#             if ('NNG' in w or 'NNP' in w or 'VA' in w or 'VV' in w)]
    return pos_text

while True:
    input_text = str(input("Input:"))
    if input_text == "exit":
        print("Bye")
        break
    print(save_keyword(input_text))
    

Input:팔고싶어
['팔', '고', '싶', '어']
Input:팔기
['팔', '기']
Input:판매하기
['판매', '하', '기']
Input:팔구싶어
['팔', '구', '싶', '어']
Input:exit
Bye


In [38]:
tokens = [('세상', 'NNG'), ('노인', 'NNG'), ('사람', 'NNG'), ('키', 'NNG'), ('설정', 'NNG'), ('처음', 'NNG')]
list(filter(lambda w: 'NNG' in w or 'NNP' in w, tokens))
list(map(lambda w: w[0], tokens))

['세상', '노인', '사람', '키', '설정', '처음']

In [56]:
items = w2v_model.wv.most_similar("유모차")
items

[('디럭스', 0.7803670167922974),
 ('절충', 0.7298730611801147),
 ('페도라', 0.7296686172485352),
 ('TG', 0.7134497165679932),
 ('휴대용', 0.7102293372154236),
 ('타보', 0.7077086567878723),
 ('퀴드', 0.7064352631568909),
 ('기내', 0.6888257265090942),
 ('쇼퍼', 0.681729793548584),
 ('스토케', 0.6809173822402954)]

In [62]:
items = ["유모차", "카시트", "중고", "거래"]
items = list(filter(lambda noun: 0 \
                    if noun == "중고" or \
                       noun == "판매" or \
                       noun == "거래" else noun, items))
items

['유모차', '카시트']